<a href="https://colab.research.google.com/github/GuangShuaiWang/Colabnotebook/blob/main/pytorch-quickstart_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline

来源：https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

Quickstart
===================
Most machine learning workflows involve working with data, creating models, optimizing model parameters, and saving the trained models. （这句话概括的非常的好，基本上就是现在的工作思路）

本节的内容也是按照上面的四部分展开的。




In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

首先是引入必要的模块，Pytorch的模块名是torch，nn中好像是包含了构建模型的内容，这里有个重点是，在构建模型的时候要重写`nn.model`的__init__和forward两个内容。

pytorch中具有自己的分类数据库，torchtext，torchvision之类的，用于保存类型的数据。可以从datasets中下载。

Dataloader是将数据变化成可迭代的对象，用于后续的训练。ToTensor我理解是转换为tensortensor。

In [3]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)
# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)
#这里还是有一些很奇特的参数的

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [13]:
training_data
#而且通过输出可以看出，datasets也是pytorch中的一个类，.shape属性没有意义。可能只是对于它自己的数据，如果是用户自己的数据，可能还是跟tensorflow一样。

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

We pass the ``Dataset`` as an argument to ``DataLoader``. This wraps an iterable over our dataset, and supports
automatic batching, sampling, shuffling and multiprocess data loading. Here we define a batch size of 64, i.e. each element
in the dataloader iterable will return a batch of 64 features and labels.

DataLoader可以将数据按照batch_size的大小形成迭代器



In [7]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break
#这样就可以看到一个batch_size的大小了

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


--------------




Creating Models
------------------
To define a neural network in PyTorch, we create a class that inherits
from `nn.Module <https://pytorch.org/docs/stable/generated/torch.nn.Module.html>`_. We define the layers of the network
in the ``__init__`` function and specify how data will pass through the network in the ``forward`` function. To accelerate
operations in the neural network, we move it to the GPU if available.

上面已经解释的听清楚了，新的模型需要继承nn.Module类，并且要重写两个部分.


In [8]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
# 指出可用的设备为什么，cpu和gpu这两个参数在后面的模型训练时也是一个很重要的参数

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__() #继承之前模型的属性和方法，下面则是构建自己模型的属性和方法
        self.flatten = nn.Flatten()
        #下面则是模型架构，这里的构造方式是值得看看的
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        #模型的训练过程
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


--------------




Optimizing the Model Parameters
----------------------------------------
To train a model, we need a `loss function <https://pytorch.org/docs/stable/nn.html#loss-functions>`_
and an `optimizer <https://pytorch.org/docs/stable/optim.html>`_.

接下来就需要指定损失函数和优化器了，


In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In a single training loop, the model makes predictions on the training dataset (fed to it in batches), and
backpropagates the prediction error to adjust the model's parameters.

backpropagates：反向传播


In [10]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

上面的代码给我的总体感觉就是很散装，不像tensorflow那样那么集成，

We also check the model's performance against the test dataset to ensure it is learning.



In [11]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

The training process is conducted over several iterations (*epochs*). During each epoch, the model learns
parameters to make better predictions. We print the model's accuracy and loss at each epoch; we'd like to see the
accuracy increase and the loss decrease with every epoch.



In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.297143  [    0/60000]
loss: 2.285239  [ 6400/60000]
loss: 2.263265  [12800/60000]
loss: 2.267346  [19200/60000]
loss: 2.248002  [25600/60000]
loss: 2.221323  [32000/60000]
loss: 2.227829  [38400/60000]
loss: 2.191625  [44800/60000]
loss: 2.180259  [51200/60000]
loss: 2.168399  [57600/60000]
Test Error: 
 Accuracy: 39.7%, Avg loss: 2.154086 

Epoch 2
-------------------------------
loss: 2.151702  [    0/60000]
loss: 2.146778  [ 6400/60000]
loss: 2.086222  [12800/60000]
loss: 2.114624  [19200/60000]
loss: 2.064134  [25600/60000]
loss: 1.998835  [32000/60000]
loss: 2.025651  [38400/60000]
loss: 1.945483  [44800/60000]
loss: 1.941683  [51200/60000]
loss: 1.885597  [57600/60000]
Test Error: 
 Accuracy: 52.9%, Avg loss: 1.883512 

Epoch 3
-------------------------------
loss: 1.901408  [    0/60000]
loss: 1.881854  [ 6400/60000]
loss: 1.763220  [12800/60000]
loss: 1.814473  [19200/60000]
loss: 1.709618  [25600/60000]
loss: 1.656201  [32000/600

Read more about `Training your model <optimization_tutorial.html>`_.




--------------




后面就是保存和读取模型的内容了，使用到的场景并不多。

Saving Models
-------------
A common way to save a model is to serialize the internal state dictionary (containing the model parameters).



In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Loading Models
----------------------------

The process for loading a model includes re-creating the model structure and loading
the state dictionary into it.



In [ ]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

This model can now be used to make predictions.



In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Read more about `Saving & Loading your model <saveloadrun_tutorial.html>`_.


